<a href="https://colab.research.google.com/github/arita37/time/blob/master/feature_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd 'drive'

/content/drive


In [ ]:
%cd 'My Drive/Upwork'

/content/drive/My Drive/Upwork


In [ ]:
import numpy as np
import pandas as pd

import time

import os
import matplotlib.pyplot as plt
plt.style.use("dark_background")
import seaborn as sns

# Time series analysis
import statsmodels.api as sm
from pandas.plotting import autocorrelation_plot

# Data preprocessing
from sklearn.model_selection import train_test_split

# Library
import lightgbm as lgb

# Grid search
from sklearn.model_selection import GridSearchCV

# Validataion
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
train_df = pd.read_csv("sales_train_validation.csv")
calendar_df = pd.read_csv("calendar.csv")
price_df = pd.read_csv("sell_prices.csv")
sample = pd.read_csv("sample_submission.csv")

In [ ]:
calendar_df["date_dt"] = pd.to_datetime(calendar_df["date"])

In [ ]:
print("train data:{}".format(train_df.shape))
print("calendar data:{}".format(calendar_df.shape))
print("price data:{}".format(price_df.shape))
print("sample data:{}".format(sample.shape))

train data:(30490, 1919)
calendar data:(1969, 15)
price data:(6841121, 4)
sample data:(60980, 29)


In [ ]:
train  = train_df.copy()
price = price_df.copy()
calendar = calendar_df.copy()

In [ ]:
def lag_featrues(df):
    out_df = df[['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']]
    ###############################################################################
    # day lag 29~57 day and last year's day lag 1~28 day 
    day_lag = df.iloc[:,-28:]
    day_year_lag = df.iloc[:,-393:-365]
    day_lag.columns = [str("lag_{}_day".format(i)) for i in range(29,57)] # Rename columns
    day_year_lag.columns = [str("lag_{}_day_of_last_year".format(i)) for i in range(1,29)]
    
    # Rolling mean(3) and (7) and (28) and (84) 29~57 day and last year's day lag 1~28 day 
    rolling_3 = df.iloc[:,-730:].T.rolling(3).mean().T.iloc[:,-28:]
    rolling_3.columns = [str("rolling3_lag_{}_day".format(i)) for i in range(29,57)] # Rename columns
    rolling_3_year = df.iloc[:,-730:].T.rolling(3).mean().T.iloc[:,-393:-365]
    rolling_3_year.columns = [str("rolling3_lag_{}_day_of_last_year".format(i)) for i in range(1,29)]
    
    rolling_7 = df.iloc[:,-730:].T.rolling(7).mean().T.iloc[:,-28:]
    rolling_7.columns = [str("rolling7_lag_{}_day".format(i)) for i in range(29,57)] # Rename columns
    rolling_7_year = df.iloc[:,-730:].T.rolling(7).mean().T.iloc[:,-393:-365]
    rolling_7_year.columns = [str("rolling7_lag_{}_day_of_last_year".format(i)) for i in range(1,29)]
    
    rolling_28 = df.iloc[:,-730:].T.rolling(28).mean().T.iloc[:,-28:]
    rolling_28.columns = [str("rolling28_lag_{}_day".format(i)) for i in range(29,57)]
    rolling_28_year = df.iloc[:,-730:].T.rolling(28).mean().T.iloc[:,-393:-365]
    rolling_28_year.columns = [str("rolling28_lag_{}_day_of_last_year".format(i)) for i in range(1,29)]
    
    rolling_84 = df.iloc[:,-730:].T.rolling(84).mean().T.iloc[:,-28:]
    rolling_84.columns = [str("rolling84_lag_{}_day".format(i)) for i in range(29,57)]
    rolling_84_year = df.iloc[:,-730:].T.rolling(84).mean().T.iloc[:,-393:-365]
    rolling_84_year.columns = [str("rolling84_lag_{}_day_of_last_year".format(i)) for i in range(1,29)]
    
    # monthly lag 1~18 month
    month_lag = pd.DataFrame({})
    for i in range(1,19):
        if i == 1:
            monthly = df.iloc[:,-28*i:].T.sum().T
            month_lag["monthly_lag_{}_month".format(i)] = monthly
        else:
            monthly = df.iloc[:, -28*i:-28*(i-1)].T.sum().T
            month_lag["monthly_lag_{}_month".format(i)] = monthly
            
    # combine day lag and monthly lag
    out_df = pd.concat([out_df, day_lag], axis=1)
    out_df = pd.concat([out_df, day_year_lag], axis=1)
    out_df = pd.concat([out_df, rolling_3], axis=1)
    out_df = pd.concat([out_df, rolling_3_year], axis=1)
    out_df = pd.concat([out_df, rolling_7], axis=1)
    out_df = pd.concat([out_df, rolling_7_year], axis=1)
    out_df = pd.concat([out_df, rolling_28], axis=1)
    out_df = pd.concat([out_df, rolling_28_year], axis=1)
    out_df = pd.concat([out_df, rolling_84], axis=1)
    out_df = pd.concat([out_df, rolling_84_year], axis=1)
    out_df = pd.concat([out_df, month_lag], axis=1)
    
    ###############################################################################
    # dept_id
    group_dept = df.groupby("dept_id").sum()
    # day lag 29~57 day and last year's day lag 1~28 day 
    dept_day_lag = group_dept.iloc[:,-28:]
    dept_day_year_lag = group_dept.iloc[:,-393:-365]
    dept_day_lag.columns = [str("dept_lag_{}_day".format(i)) for i in range(29,57)]
    dept_day_year_lag.columns = [str("dept_lag_{}_day_of_last_year".format(i)) for i in range(1,29)]
    # monthly lag 1~18 month
    month_dept_lag = pd.DataFrame({})
    for i in range(1,19):
        if i == 1:
            monthly_dept = group_dept.iloc[:,-28*i:].T.sum().T
            month_dept_lag["dept_monthly_lag_{}_month".format(i)] = monthly_dept
        elif i >= 7 and i < 13:
            continue
        else:
            monthly = group_dept.iloc[:, -28*i:-28*(i-1)].T.sum().T
            month_dept_lag["dept_monthly_lag_{}_month".format(i)] = monthly_dept
    # combine out df
    out_df = pd.merge(out_df, dept_day_lag, left_on="dept_id", right_index=True, how="left")
    out_df = pd.merge(out_df, dept_day_year_lag, left_on="dept_id", right_index=True, how="left")
    out_df = pd.merge(out_df, month_dept_lag, left_on="dept_id", right_index=True, how="left")
    
    ###############################################################################       
    # cat_id
    group_cat = df.groupby("cat_id").sum()
    # day lag 29~57 day and last year's day lag 1~28 day 
    cat_day_lag = group_cat.iloc[:,-28:]
    cat_day_year_lag = group_cat.iloc[:,-393:-365]
    cat_day_lag.columns = [str("cat_lag_{}_day".format(i)) for i in range(29,57)]
    cat_day_year_lag.columns = [str("cat_lag_{}_day_of_last_year".format(i)) for i in range(1,29)]
    # monthly lag 1~18 month
    month_cat_lag = pd.DataFrame({})
    for i in range(1,19):
        if i == 1:
            monthly_cat = group_cat.iloc[:,-28*i:].T.sum().T
            month_cat_lag["cat_monthly_lag_{}_month".format(i)] = monthly_cat
        elif i >= 7 and i < 13:
            continue
        else:
            monthly_cat = group_cat.iloc[:, -28*i:-28*(i-1)].T.sum().T
            month_cat_lag["dept_monthly_lag_{}_month".format(i)] = monthly_cat
            
    # combine out df
    out_df = pd.merge(out_df, cat_day_lag, left_on="cat_id", right_index=True, how="left")
    out_df = pd.merge(out_df, cat_day_year_lag, left_on="cat_id", right_index=True, how="left")
    out_df = pd.merge(out_df, month_cat_lag, left_on="cat_id", right_index=True, how="left")
    
    ###############################################################################
    # store_id
    group_store = df.groupby("store_id").sum()
    # day lag 29~57 day and last year's day lag 1~28 day 
    store_day_lag = group_store.iloc[:,-28:]
    store_day_year_lag = group_store.iloc[:,-393:-365]
    store_day_lag.columns = [str("store_lag_{}_day".format(i)) for i in range(29,57)]
    store_day_year_lag.columns = [str("store_lag_{}_day_of_last_year".format(i)) for i in range(1,29)]
    # monthly lag 1~18 month
    month_store_lag = pd.DataFrame({})
    for i in range(1,19):
        if i == 1:
            monthly_store = group_store.iloc[:,-28*i:].T.sum().T
            month_store_lag["store_monthly_lag_{}_month".format(i)] = monthly_store
        elif i >= 7 and i <13:
            continue
        else:
            monthly_store = group_store.iloc[:, -28*i:-28*(i-1)].T.sum().T
            month_store_lag["store_monthly_lag_{}_month".format(i)] = monthly_store
            
    # combine out df
    out_df = pd.merge(out_df, store_day_lag, left_on="store_id", right_index=True, how="left")
    out_df = pd.merge(out_df, store_day_year_lag, left_on="store_id", right_index=True, how="left")
    out_df = pd.merge(out_df, month_store_lag, left_on="store_id", right_index=True, how="left")
    
    ###############################################################################
    # state_id
    group_state = df.groupby("state_id").sum()
    # day lag 29~57 day and last year's day lag 1~28 day 
    state_day_lag = group_state.iloc[:,-28:]
    state_day_year_lag = group_state.iloc[:,-393:-365]
    state_day_lag.columns = [str("state_lag_{}_day".format(i)) for i in range(29,57)]
    state_day_year_lag.columns = [str("state_lag_{}_day_of_last_year".format(i)) for i in range(1,29)]
    # monthly lag 1~18 month
    month_state_lag = pd.DataFrame({})
    for i in range(1,13):
        if i == 1:
            monthly_state = group_state.iloc[:,-28*i:].T.sum().T
            month_state_lag["state_monthly_lag_{}_month".format(i)] = monthly_state
        elif i >= 7 and i < 13:
            continue
        else:
            monthly_state = group_state.iloc[:, -28*i:-28*(i-1)].T.sum().T
            month_state_lag["state_monthly_lag_{}_month".format(i)] = monthly_state
            
    # combine out df
    out_df = pd.merge(out_df, state_day_lag, left_on="state_id", right_index=True, how="left")
    out_df = pd.merge(out_df, state_day_year_lag, left_on="state_id", right_index=True, how="left")
    out_df = pd.merge(out_df, month_state_lag, left_on="state_id", right_index=True, how="left")
    
    ###############################################################################
    # category flag
    col_list = ['dept_id', 'cat_id', 'store_id', 'state_id']
    
    df_cate_oh = pd.DataFrame({})
    for i in col_list:
        df_oh = pd.get_dummies(df[i])
        df_cate_oh = pd.concat([df_cate_oh, df_oh], axis=1)
        
    out_df = pd.concat([out_df, df_cate_oh], axis=1)
    
    return out_df

In [ ]:
import pandas as pd

#df_metrics.to_csv("eval_stats.csv", sep="\t")
#df_metrics.head()
df_metrics={'run_id':[], 'feat_name':[], "feat_name_long":[], "feat_type":[], "model_params":[], "metric_name":[], "metric_val":[]}
df_metrics = pd.DataFrame(data=df_metrics)
df_metrics.columns


Index(['run_id', 'feat_name', 'feat_name_long', 'feat_type', 'model_params',
       'metric_name', 'metric_val'],
      dtype='object')

In [ ]:
# Features
Train_data = train.iloc[:,:-56]
Val_data = train.iloc[:,:-28]
pred=[]
X_train = lag_featrues(Train_data).iloc[:,5:] # select variables
y_train = train.iloc[:,-56]
X_test = lag_featrues(Val_data).iloc[:,5:]
y_test = train.iloc[:,-28]

# Create instance
lgbm = lgb.LGBMRegressor()

# Training and score
learning_rate = [0.15, 0.2, 0.25]
max_depth = [15, 20, 25]

param_grid = {'learning_rate': learning_rate, 'max_depth': max_depth}
param=param_grid
#df_metrics.model_params=max_depth
# Fitting
cv_lgbm = GridSearchCV(lgbm, param_grid, cv=10, n_jobs =1)
cv_lgbm.fit(X_train, y_train)

print("Best params:{}".format(cv_lgbm.best_params_))

# best params
best_lg = cv_lgbm.best_estimator_

# prediction
y_train_pred_lg = best_lg.predict(X_train)
y_test_pred_lg = best_lg.predict(X_test)

print("MSE train:{}".format(mean_squared_error(y_train, y_train_pred_lg)))
print("MSE test;{}".format(mean_squared_error(y_test, y_test_pred_lg)))
pred_mse=y_train_pred_lg
print("R2 score train:{}".format(r2_score(y_train, y_train_pred_lg)))
pred_rmse=y_train_pred_lg
print("R2 score test:{}".format(r2_score(y_test, y_test_pred_lg)))
df_metrics.metric_name="RMSE"

Best params:{'learning_rate': 0.15, 'max_depth': 15}
MSE train:1.092012714799604
MSE test;3.3766285234516995
R2 score train:0.880892037264018
R2 score test:0.6074382220106134


In [ ]:

#best_lg="boosting_type='gbdt',learning_rate=0.15,"

#dataframe['parameter'] = X_train.columns


In [ ]:
#Predict using only variables with an importance of 1 or higher.
importance = best_lg.feature_importances_

indices = np.argsort(importance)[::-1]

# print importance
importance_df = pd.DataFrame({})
columns = []
importance_ = []
for f in range(X_train.shape[1]):
    print("%2d) %-*s %.2f" %(f+1, 30, X_train.columns[indices[f]], importance[indices[f]]))
    col = X_train.columns[indices[f]]
    imp = importance[indices[f]]
    columns.append(col)
    importance_.append(imp)
importance_df["col_name"] = columns
importance_df["importance"] = importance_
importance = best_lg.feature_importances_

indices = np.argsort(importance)[::-1]



 1) lag_44_day                     74.00
 2) rolling3_lag_56_day            65.00
 3) rolling7_lag_56_day            43.00
 4) lag_56_day                     40.00
 5) rolling3_lag_46_day            34.00
 6) rolling3_lag_33_day            33.00
 7) lag_32_day                     32.00
 8) lag_33_day                     32.00
 9) monthly_lag_10_month           30.00
10) rolling28_lag_56_day           29.00
11) lag_55_day                     29.00
12) lag_54_day                     29.00
13) rolling3_lag_51_day            28.00
14) monthly_lag_3_month            28.00
15) rolling3_lag_45_day            28.00
16) monthly_lag_8_month            26.00
17) lag_29_day                     25.00
18) monthly_lag_18_month           24.00
19) rolling3_lag_47_day            24.00
20) lag_34_day                     24.00
21) monthly_lag_7_month            24.00
22) rolling3_lag_44_day            23.00
23) lag_43_day                     23.00
24) rolling3_lag_52_day            23.00
25) monthly_lag_

In [ ]:
df_metrics=df_metrics[:366]
df_metrics

,run_id,feat_name,feat_name_long,feat_type,model_params,metric_name,metric_val


In [ ]:
X_train.columns[0:364]
df_metrics.feat_name


Series([], Name: feat_name, dtype: float64)

In [ ]:
df_metrics.feat_name=X_train.columns[0:365]
df_metrics.metric_name="RMSE"
df_metrics.feat_type=df_metrics.feat_name
#df_metrics['feat_type'].replace(to_replace=r'^lag.$', value='hj', regex=True,inplace=True)
df_metrics.replace({'feat_type': r'^lag_.*'}, {'feat_type': 'lag'}, regex=True,inplace=True)
df_metrics.replace({'feat_type': r'^rolling_.*'}, {'feat_type': 'rolling'}, regex=True,inplace=True)
#indices=df_metrics.feat_name.str.find("lag")
#df_metrics["feat_type"]=df_metrics.feat_name.str.find("lag")
#df_metrics["feature_type"]=indices


In [ ]:
# importance columns (>0)
imp_col = importance_df[importance_df["importance"]>0]["col_name"].values

# Train test split, select by imp_col

X_train = lag_featrues(Train_data).iloc[:,5:][imp_col] # select variables
y_train = train.iloc[:,-56]
X_test = lag_featrues(Val_data).iloc[:,5:][imp_col]
y_test = train.iloc[:,-28]

# Create instance
lgbm = lgb.LGBMRegressor()

# Training and score
learning_rate = [0.15, 0.2, 0.25]
max_depth = [15, 20, 25]

param_grid = {'learning_rate': learning_rate, 'max_depth': max_depth}

# Fitting
cv_lgbm = GridSearchCV(lgbm, param_grid, cv=10, n_jobs =1)
cv_lgbm.fit(X_train, y_train)

print("Best params:{}".format(cv_lgbm.best_params_))

# best params
best_lg = cv_lgbm.best_estimator_

# prediction
y_train_pred_lg = best_lg.predict(X_train)
y_test_pred_lg = best_lg.predict(X_test)

print("MSE train:{}".format(mean_squared_error(y_train, y_train_pred_lg)))
print("MSE test;{}".format(mean_squared_error(y_test, y_test_pred_lg)))

print("R2 score train:{}".format(r2_score(y_train, y_train_pred_lg)))
print("R2 score test:{}".format(r2_score(y_test, y_test_pred_lg)))

Best params:{'learning_rate': 0.15, 'max_depth': 25}
MSE train:1.083809003506013
MSE test;3.3448334245952154
R2 score train:0.8817868321009381
R2 score test:0.6111346726126745


In [ ]:
run_id=list(range(300))
df_metrics=pd.DataFrame(run_id,columns=['run_id'])
df_metrics['feat_name'] = pd.Series(X_train.columns[0:300], index=dataframe.index) 
df_metrics['feat_type']=df_metrics.feat_name
df_metrics.replace({'feat_type': r'^lag_.*'}, {'feat_type': 'lag'}, regex=True,inplace=True)
df_metrics.replace({'feat_type': r'^rolling.*'}, {'feat_type': 'rolling'}, regex=True,inplace=True)
df_metrics['parameter'] = pd.Series(best_lg, index=dataframe.index) 
df_metrics['metric_name'] ="MSE" 
df_metrics['metric_val'] = pd.Series(pred_mse[:300], index=dataframe.index) 
df_metrics.to_csv("train.csv")


df_metrics

,run_id,feat_name,feat_type,parameter,metric_name,metric_val
0,0,lag_44_day,lag,"LGBMRegressor(boosting_type='gbdt', class_weig...",MSE,0.836521
1,1,rolling3_lag_56_day,rolling,"LGBMRegressor(boosting_type='gbdt', class_weig...",MSE,0.220465
2,2,rolling7_lag_56_day,rolling,"LGBMRegressor(boosting_type='gbdt', class_weig...",MSE,0.211865
3,3,lag_56_day,lag,"LGBMRegressor(boosting_type='gbdt', class_weig...",MSE,1.901740
4,4,rolling3_lag_46_day,rolling,"LGBMRegressor(boosting_type='gbdt', class_weig...",MSE,0.841908
...,...,...,...,...,...,...
295,295,rolling84_lag_15_day_of_last_year,rolling,"LGBMRegressor(boosting_type='gbdt', class_weig...",MSE,0.214783
296,296,rolling84_lag_45_day,rolling,"LGBMRegressor(boosting_type='gbdt', class_weig...",MSE,0.103399
297,297,store_lag_9_day_of_last_year,store_lag_9_day_of_last_year,"LGBMRegressor(boosting_type='gbdt', class_weig...",MSE,0.167058
298,298,HOBBIES_1,HOBBIES_1,"LGBMRegressor(boosting_type='gbdt', class_weig...",MSE,0.219598
